In [1]:
import gurobipy as gp
import itertools
from gurobipy import GRB

In [2]:
def tsp_solver(distances):
    num_cities = len(distances)

    # Crear el modelo
    model = gp.Model()

    # Crear las variables
    x = {}
    for i in range(num_cities):
        for j in range(num_cities):
            x[i, j] = model.addVar(vtype=GRB.BINARY, name=f'x_{i}_{j}')

    # Restricción de visita
    for i in range(num_cities):
        model.addConstr(gp.quicksum(x[i, j] for j in range(num_cities) if j != i) == 1, name=f'visit_{i}')

    for j in range(num_cities):
        model.addConstr(gp.quicksum(x[i, j] for i in range(num_cities) if i != j) == 1, name=f'visit_{j}')

    # Restricción de subtours
    for subset_size in range(2, num_cities):
        for subset in itertools.combinations(range(num_cities), subset_size):
            model.addConstr(gp.quicksum(x[i, j] for i in subset for j in range(num_cities) if j not in subset) >= 1,
                            name=f'subtour_{subset}')

    # Función objetivo
    obj = gp.quicksum(distances[i][j] * x[i, j] for i in range(num_cities) for j in range(num_cities) if j != i)
    model.setObjective(obj, GRB.MINIMIZE)

    # Resolver el modelo
    model.optimize()

    # Obtener la solución
    if model.status == GRB.OPTIMAL:
        route = []
        for i in range(num_cities):
            for j in range(num_cities):
                if x[i, j].x > 0.5:
                    route.append((i, j))
        return route
    else:
        return None

In [3]:
distances = [
    [0, 10, 15, 20],
    [10, 0, 35, 25],
    [15, 35, 0, 30],
    [20, 25, 30, 0]
]
route = tsp_solver(distances)
print("Ruta óptima:")
for i, j in route:
    print(f"De ciudad {i} a ciudad {j}")

Restricted license - for non-production use only - expires 2024-10-28
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i5-7400 CPU @ 3.00GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 18 rows, 16 columns and 60 nonzeros
Model fingerprint: 0x8df488cf
Variable types: 0 continuous, 16 integer (16 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 95.0000000
Presolve removed 4 rows and 4 columns
Presolve time: 0.03s
Presolved: 14 rows, 12 columns, 48 nonzeros
Variable types: 0 continuous, 12 integer (12 binary)

Root relaxation: objective 8.000000e+01, 7 iterations, 0.02 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent 